In [6]:
from parse_data import get_data, get_modified_values, get_binary_values, make_data_scalar
from rnn_layers import PredictTime
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device: ", device)

df = get_data()

Using device:  cuda


In [7]:

def make_data(df, device):

    x_train, y_train = [], []
    prev = None

    for row in df:
        if prev is None:
            prev = row
        x_train.append(prev)
        y_train.append(row)
        prev = row
    print(x_train[0].size())
    return torch.stack(x_train).float().to(device),torch.stack(y_train).float().to(device)

make_data(get_binary_values(get_data()), device)

print(make_data_scalar(get_data(), device))

torch.Size([1])
Max value:  1000
(tensor([[-1.0000],
        [ 1.0000],
        [ 0.4600],
        ...,
        [ 1.0000],
        [ 0.4600],
        [ 0.5900]], device='cuda:0'), tensor([[1.0000],
        [0.4600],
        [1.0000],
        ...,
        [0.4600],
        [0.5900],
        [0.5900]], device='cuda:0'))


In [ ]:
import numpy as np
import torch.optim as optim
import torch.utils.data as data
from IPython.display import clear_output
import torch.nn as nn
import random

batch_size = 100
x_d, y_d = make_data_scalar(get_data(), device)

print(y_d.size(), x_d.size())

model = PredictTime(input_size=x_d[0].size()[0],
                    output_size=y_d[0].size()[0],
                    hidden_layers=3,
                    hidden_size=50, device=device).to(device)
optimizer = optim.Adam(model.parameters(),lr=0.01)

loss = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(x_d,y_d), batch_size=batch_size)
epochs = 5000
for e in range(epochs):
    model.train()
    #print(next(iter(loader)))
    model.clean_state()
    res = []
    
    
    #if random.random() < e/(epochs*2):
        #model.teacher_forcing = False
    #else:
        #model.teacher_forcing = True
    for x, y in loader:
        
        if random.random() < 0.5:
            continue
        model.random_state()
        
        
        y_pred = model(x)
        l = loss(y_pred, y)
        res.append(l)
        #print(y_pred, y)
    
    
    l = res[0]
    for i in res[1:]:
        l += i
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    if e % 10 != 0:
        continue
    #clear_output(wait=True)
    sum_loss = 0
    #print(list(model.parameters())[-1])

    for x, y in loader:
        model.eval()
        with torch.no_grad():
            y_pred = model(x)
            sum_loss += np.sqrt(loss(y_pred, y).cpu())
    
    print("Epoch %d Loss %.4f" % (e, sum_loss))
    
    
#for d in df.values:

Max value:  1000
torch.Size([8152, 1]) torch.Size([8152, 1])
Epoch 0 Loss 29.8433
Epoch 10 Loss 29.7095
Epoch 20 Loss 29.6214
Epoch 30 Loss 29.5551
Epoch 40 Loss 29.2185
Epoch 50 Loss 28.5259
Epoch 60 Loss 28.4496
Epoch 70 Loss 28.4447
Epoch 80 Loss 28.4022
Epoch 90 Loss 28.3774
Epoch 100 Loss 28.3584
Epoch 110 Loss 28.3727
Epoch 120 Loss 28.3469
Epoch 130 Loss 28.3149
Epoch 140 Loss 28.3378
Epoch 150 Loss 28.3100
Epoch 160 Loss 28.3131
Epoch 170 Loss 28.2708
Epoch 180 Loss 28.2140
Epoch 190 Loss 28.1580
Epoch 200 Loss 28.1284
Epoch 210 Loss 28.0243
Epoch 220 Loss 27.9481
Epoch 230 Loss 27.8191
Epoch 240 Loss 27.5741
Epoch 250 Loss 27.4800
Epoch 260 Loss 27.2817
Epoch 270 Loss 27.0101
Epoch 280 Loss 26.4766
Epoch 290 Loss 25.9748
Epoch 300 Loss 26.2202
Epoch 310 Loss 25.2728
Epoch 320 Loss 24.4186
Epoch 330 Loss 23.5503
Epoch 340 Loss 22.9343
Epoch 350 Loss 23.8585
Epoch 360 Loss 22.3273
Epoch 370 Loss 21.4727
Epoch 380 Loss 20.6055
Epoch 390 Loss 19.8990
Epoch 400 Loss 19.8815
Epoch 4

In [ ]:
import matplotlib.pyplot as plt

def eval_model(x,y,m):
    df = get_data() 
    maxtime = df.max()[1]

    res = []
    m.eval()
    m.clean_state()
    count  = 1
    prev = x[0][0]
    for i in x:
        t = torch.tensor([[prev]]).to(device)
        prev = i
        val = m(t)
        count += 1
        res.append(val.detach().cpu()[0])
    
    loss = nn.MSELoss()
    print(np.sqrt(loss(torch.tensor(res).to(device),y.squeeze()).cpu()))
    

    fig, ax = plt.subplots(2)
    
    ax[0].plot(range(1,51), res[:50])
    ax[0].plot( range(1,51), y[:50].cpu())

    ax[1].plot(range(2000,2050), res[2000:2050])
    
    ax[1].plot(range(2000,2050), y[2000:2050].cpu() )
    fig.suptitle("Result when feeding correct values as input")
    fig.savefig("teacher_forcing.png")


amount = 1000
eval_model(x_d,y_d,model)


In [ ]:
import matplotlib.pyplot as plt

def eval_model(x,y,m):
    df = get_data() 
    maxtime = df.max()[1]

    res = []
    m.eval()
    m.clean_state()
    count  = 1
    prev = x[0][0]
    for i in x:
        t = torch.tensor([[prev]]).to(device)
        prev = i
        val = m(t)
        prev = val
        count += 1
        res.append(val.detach().cpu()[0])
    
    loss = nn.MSELoss()
    print(np.sqrt(loss(torch.tensor(res).to(device),y.squeeze()).cpu()))
    
    fig, ax = plt.subplots(2)
    
    ax[0].plot(range(1,51), res[:50])
    ax[0].plot( range(1,51), y[:50].cpu())

    ax[1].plot(range(2000,2050), res[2000:2050])
    
    ax[1].plot(range(2000,2050), y[2000:2050].cpu() )
    fig.suptitle("Result without feeding correct values as input")
    fig.savefig("teacher_forcing_not_correct.png")
    
amount = 1000
eval_model(x_d,y_d,model)
